<a href="https://colab.research.google.com/github/fdabrandao/MO-book-with-AMPL/blob/dev/notebooks/01/production-planning-advanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A Data-Driven AMPL Model

In this notebook, we'll revisit the production planning example. However, this time we'll demonstrate how Python's data structures combine with AMPL's ability to separate model and data, to create an optimization model that scales with the size of the data tables. This enables the model to adjust to new products, varying prices, or changing demand. We refer to this as "data-driven" modeling.

This notebook introduces two new AMPL model components that describe the data in a general way:

* Sets
* Parameters

These components enable the model to specify variables, constraints, and summations that are *indexed* over sets. The combination of sets and indices is essential to building scalable and maintainable models for more complex applications.

We will begin this analysis by examining the production planning data sets to identify the underlying problem structure. Then we will reformulate the mathematical model in a more general way that is valid for any data scenario. Finally we show how the same formulation carries over naturally into AMPL, providing a clear, data-driven formulation of the production planning application.

In [ ]:
# install dependencies and select solver
!pip install -q amplpy matplotlib pandas

## Data representations

We begin by revisiting the data tables and mathematical model developed for the basic production planning problem presented in the previous notebook. The original data values were given as follows:

| Product | Material <br> required | Labor A <br> required | Labor B <br> required | Market <br> Demand | Price |
| :-: | :-: | :-: | :-: | :-: | :-: |
| U | 10 g | 1 hr | 2 hr | $\leq$ 40 units | \$270 |
| V |  9 g | 1 hr | 1 hr | unlimited | \$210 |

| Resource | Amount <br> Available | Cost |
| :-: | :-: | :-: |
| Material | ? | \$10 / g |
| Labor A | 80 hours | \$50 / hour |
| Labor B | 100 hours | \$40 / hour |

Two distinct *sets* of objects are evident from these tables. The first is the set of products comprised of $U$ and $V$. The second is the set of resources used to produce those products, which we have abbreviated as $M$, $A$, and $B$.

Having identified these sets, the data for this application be factored into three simple tables. The first two tables list attributes of the products and attributes of the resources. The third table summarizes the processes used to create the products from the resources, which requires providing a value for each combination of product and resource:

**Table: Products**

| Product | Demand | Price |
| :-: | :-: | :-: |
| U |  $\leq$ 40 units | \$270 |
| V |  unlimited | \$210 |

**Table: Resources**

| Resource | Available | Cost |
| :-: | :-: | :-: |
| M | ? | \$10 / g |
| A | 80 hours | \$50 / hour |
| B | 100 hours | \$40 / hour |

**Table: Processes**

| Product | M | A | B
| :-: | :-: | :-: | :-: |
| U | 10 g | 1 hr | 2 hr |
| V |  9 g | 1 hr | 1 hr |


Python has many built-in data types and libraries that are useful for handling tabular data, and there are several options that would be appropriate for the task at hand. Nested dictionaries can be a good choice for data of moderate size. In the following examples, we will show how nested dictionaries can be used to represent the three tables described above.

The first table describes the products. The product names serve as keys for outermost dictionary, and attribute names as keys for the inner dictionaries. The attribute values are numbers; `None` is specified when a value is not present.

In [ ]:
products = {
    "U": {"demand": 40, "price": 270},
    "V": {"demand": None, "price": 210},
}

# print data
for product, attributes in products.items():
    for attribute, value in attributes.items():
        print(f"{product} {attribute:10s} {value}")

U demand     40
U price      270
V demand     None
V price      210


The second table is the nested dictionary listing attributes and values for resources consumed.

In [ ]:
resources = {
    "M": {"available": None, "cost": 10},
    "A": {"available":   80, "cost": 50},
    "B": {"available":  100, "cost": 40},
}

for resource, attributes in resources.items():
    for attribute, value in attributes.items():
        print(f"{resource:8s} {attribute:10s} {value}")

M        available  None
M        cost       10
A        available  80
A        cost       50
B        available  100
B        cost       40


The third table data shows the amount of each resource needed to produce one unit of each product. Both sets serve as keys here: the product names for outermost dictionary, and resource names for the inner dictionaries.

In [ ]:
processes = {
    "U": {"M": 10, "A": 2, "B": 1},
    "V": {"M":  9, "A": 1, "B": 1},
}

for product, process in processes.items():
    for resource, value in process.items():
        print(f"{product:4s} {resource:10s} {value}")

U    M          10
U    A          2
U    B          1
V    M          9
V    A          1
V    B          1


## Mathematical model

By rearranging the problem data into straightforward tables, the structure of the production planning problem becomes evident. We can identify a set of products, a set of resources, and a collection of parameters that specify the processes for transforming resources into products. Compared to the previous notebook, these abstractions allow us to create mathematical models that can adapt and scale with the supplied data.

Let $\cal{P}$ and $\cal{R}$ be the set of products and resources, respectively, and let $p$ and $r$ be representative elements of those sets. We use indexed decision variables $x_r$ to denote the amount of resource $r$ that is consumed in production, and $y_p$ to denote the amount of product $p$ produced.

The problem data provides attributes that constrain the decision variables. For example, the decision variables all have lower bounds of zero, and some have upper bounds. We represent these as

$$
\begin{aligned}
    0 \leq x_r \leq b^x_r & & \forall r\in\cal{R} \\
    0 \leq y_p \leq b^y_p & & \forall p\in\cal{P} \\
\end{aligned}
$$

where the upper bounds, $b^x_r$ and $b^y_p$, are data taken from the tables of attributes. For cases where the upper bounds don't exist . . . **add explanation here**

The objective is given as before,

$$
\begin{aligned}
    \text{profit} & = \text{revenue} - \text{cost} \\
\end{aligned}
$$

but now the expressions for revenue and cost are expressed more generally as sums over the product and resource sets,

$$
\begin{aligned}
    \text{revenue} & = \sum_{p\in\cal{P}} c^y_p y_p  \\
    \text{cost} & = \sum_{r\in\cal{R}} c^x_r x_r \\
\end{aligned}
$$

where parameters $c^y_p$ and $c^x_r$ represent the selling prices for products and the costs for resources, respectively. The limits on available resources can be written as

$$
\begin{aligned}
    \sum_{p\in\cal{P}} a_{rp} y_p & \leq x_r & \forall r\in\cal{R}
\end{aligned}
$$

where $a_{rp}$ is the amount of resource $r$ needed to make 1 unit of product $p$. Putting these pieces together, we have the following model for the production planning problem.

$$
\begin{align}
{\rm maximize} \quad & \sum_{p\in\cal{P}} c^y_p y_p - \sum_{r\in\cal{R}} c^x_r x_r \\
\text{subject to} \quad & \sum_{p\in\cal{P}} a_{rp} y_p  \leq x_r & \forall r\in\cal{R} \nonumber \\
 &   0 \leq x_r \leq b^x_r & \forall r\in\cal{R} \nonumber  \\
 &   0 \leq y_p \leq b^y_p & \forall p\in\cal{P} \nonumber  \\
\end{align}
$$

When formulated this way, the model can be applied to any problem with the same structure, regardless of the number of products or resources. This flexibility is possible due to the use of sets to describe the products and resources for a particular problem, indices like $p$ and $r$ to refer to elements of those sets, and data tables that hold the relevant parameter values.

Generalizing mathematical models in this fashion is a feature of all large-scale optimization applications. Next we will see how this type of generalization carries over naturally into formulating and solving the model in AMPL.

## The production model in AMPL

As before, we begin the construction of an AMPL model by importing the needed components into the AMPL environment.

In [ ]:
from amplpy import AMPL, tools

ampl = tools.ampl_notebook(
    modules=["highs"], # modules to install
    license_uuid="default", # license to use
    g=globals()) # instantiate AMPL object and register magics

Next we use AMPL `set` statements to define the product and resource sets. Notice that at this point, we are only telling AMPL about the two sets will be used in the model. The members of these sets will be sent from Python to AMPL later, as part of the problem data.

In mathematical formulations, it is customary to keep the names of all components short. But when writing the model in AMPL, we are free to use longer, more meaningful names that make the model statements easier to read. Thus, for example, here we use `PRODUCTS` and `RESOURCES` as the AMPL names of the sets that are are called $\cal P$ and $\cal R$ in the mathematical model.

In [ ]:
%%ampl_eval
# define sets

set PRODUCTS;
set RESOURCES;

The next step is to introduce parameters that will be used as data in the objective function and the constraints.

A statement that defines and AMPL parameter begin with the `param` keyword and a unique name. Then between braces `{` and `}` it specifies the index sets for the parameter. For example:

- `param demand {PRODUCTS} >= 0;` states that there is a "product demanded" value for each member of the set PRODUCTS.

- `param need {RESOURCES,PRODUCTS} >= 0;` states that there is a "resource needed" value for each combination of a resource and a product.

At the end of each `param` statement, we specify that the values for the parameter must be `> 0` or `>= 0`, as appropriate. These specifications will be used to later to check that the actual data values are appropriate for the problem.

In [ ]:
%%ampl_eval
# define parameters

param demand {PRODUCTS} >= 0;
param price {PRODUCTS} > 0;

param available {RESOURCES} >= 0;
param cost {RESOURCES} > 0;

param need {RESOURCES,PRODUCTS} >= 0;

AMPL defines the decision variables in much the same way as the parameters, but with `var` as the keyword starting the statement. We name the variables `Use` for resource use, and `Sell` for product sales.

To express the bounds on the variables in the same way as the mathematical formulation, a more general form of the AMPL statement is needed. In the case of the `Use` variables, for example:

*  The indexing expression is written `{r in RESOURCES}` to say that there is a variable for each member of the resource set, and also to associate the *index* `j` with members of the set for purpoase of this statement. This is the AMPL equivalent of $\forall r\in\cal{R}$ in the mathematical statement.

* The upper bound is written `<= available[r]` to say that for each member `r` of the resource set, the variable's upper bound is given by the corresponding value from the availability table. This is the AMPL equivalent of $\leq b^x_r$ in the mathematical statement.

An expression in brackets `[...]` is called an AMPL *subscript* because it plays the same role as a mathematical subscript like $r$ in $\leq b^x_r$. Anywhere that the model refers to particular values of an indexed parameter or variables, you will see subscript expressions. For example,

* `need[r,p]` will be the amount of resource `r` needed to make one unit of product `p`.

* `Use[r]` will be the total amount of resource `r` used.

In [ ]:
%%ampl_eval
# define variables

var Use {r in RESOURCES} >= 0, <= available[r];
var Sell {p in PRODUCTS} >= 0, <= demand[p];

Just as in the previous notebook, the AMPL statement for the objective function begins with `maximize Profit`. But now, as in the mathematical formulation, AMPL uses general summation expressions:

* `sum {p in PRODUCTS} price[p] * Sell[p]` is the sum, over all products, of the price per unit time the number sold. It corresponds to $\sum_{p\in\cal{P}} c^y_p y_p$ in the mathematical formulation.

* `sum {r in RESOURCES} cost[r] * Use[r]` is the sum, over all resources, of the cost per unit time the amount used. It corresponds to $\sum_{r\in\cal{R}} c^x_r x_r$ in the mathematical formulation.

The full expression for the objective function is simply the first of these expressions minus the second one.

In [ ]:
%%ampl_eval
# define objective function

maximize profit:
   sum {p in PRODUCTS} price[p] * Sell[p] -
   sum {r in RESOURCES} cost[r] * Use[r];

The previous AMPL model had 3 constraints, each defined by a `subject to` statement. But the data-driven mathematical formulation recognizes that there is only one kind of constraint --- resources needed must be less than or equal to resources used --- repeated 3 times, once for each resource. The AMPL version combines expressions that have already appeared in earlier parts of the model:

* `subject to ResourceLimit {r in RESOURCES}` says that the model will have one constraint corresponding to each member `r` of the resource set.

* `sum {p in PRODUCTS} need[r,p] * Sell[p] <= Use[r]` says that the total of resource `r` needed, summed over all produces sold, must be `<=` the total of resource `r` used. This corresponds to $\sum_{p\in\cal{P}} a_{rp} y_p \leq x_r$ in the mathematical formulation.

In [ ]:
%%ampl_eval
# create indexed constraint

subject to ResourceLimit {r in RESOURCES}:
   sum {p in PRODUCTS} need[r,p] * Sell[p] <= Use[r];

The final step is to solve the model and report the solution. Here we create a simple report using `pyo.value()` to access values of the decision variables, and using the model sets to construct iterators to report the value of indexed variables.

In [ ]:
# load set data
ampl.set["PRODUCTS"] = list(products.keys())
ampl.set["RESOURCES"] = list(resources.keys())

# extract data from nested dictionaries
cp = {k : v["price"] for k, v in products.items()}
demand = {k : v["demand"] for k, v in products.items()}

cr = {k : v["price"] for k, v in resources.items()}
available = {k : v["available"] for k, v in resources.items()}

a = {(k2, k) : v2 for k, v in processes.items() for k2, v2 in v.items()}

# None is interpreted as an infinite value
INF = float("inf")

for k, v in demand.items():
    if not v:
        demand[k] = INF

for k, v in available.items():
    if not v:
        available[k] = INF

# load data for parameters
ampl.param["cp"] = cp
ampl.param["demand"] = demand
ampl.param["cr"] = cr
ampl.param["available"] = available
ampl.param["a"] = a

# set solver and solve
ampl.option["solver"] = SOLVER
ampl.solve()

# create a solution report
print(f"Profit = {ampl.obj['profit'].value()}")

print("\nProduction Report")
y = ampl.var["y"].get_values()

for product in y:
    print(f" {product[0]}  produced =  {product[1]}")

print("\nResource Report")
x = ampl.var["x"].get_values()

for resource in x:
    print(f" {resource[0]} consumed = {resource[1]}")

HiGHS 1.5.1: HiGHS 1.5.1: optimal solution; objective 2400
3 simplex iterations
0 barrier iterations
Profit = 2400.0

Production Report
 U  produced =  20.0
 V  produced =  60.0

Resource Report
 M consumed = 740.0
 labor A consumed = 100.0
 labor B consumed = 80.0


## For Python experts: Creating subclasses of `AMPL`

Some readers of these notebooks may be more experienced Python developers who wish to apply Pyomo in more specialized, data driven applications. The following cell shows how the Pyomo `ConcreteModel()` class can be extended by subclassing to create specialized model classes. Here we create a subclass called `ProductionModel` that accepts a particular representation of the problem data to produce a production model object. The production model object inherits all of the methods associated with any `ConcreteModel`, such as `.display()`, `.solve()`, and `.pprint()`, but can be extended with additional methods.

In [ ]:
%%writefile production_planning.mod

# sets
set PRODUCTS;
set RESOURCES;

# parameters
param demand{PRODUCTS};
param cp{PRODUCTS};
param available{RESOURCES};
param cr{RESOURCES};
param a{RESOURCES, PRODUCTS};

# variables
var x{r in RESOURCES} >= 0, <= available[r];
var y{p in PRODUCTS} >= 0, <= demand[p];

# auxiliary variables
var revenue = sum{p in PRODUCTS} cp[p] * y[p];
var cost = sum{r in RESOURCES} cr[r] * x[r];

# objective
maximize profit: revenue - cost;

# constraints
s.t. materials_used {r in RESOURCES}:
    sum{p in PRODUCTS} a[r, p] * y[p] <= x[r];

Overwriting production_planning.mod


In [ ]:
import pandas as pd

class ProductionModel(AMPL):
    """
    A class representing a production model using AMPL.
    """

    def __init__(self, products, resources, processes):
        """
        Initialize ProductionModel as an AMPL instance.

        :param products: A dictionary containing product information.
        :param resources: A dictionary containing resource information.
        :param processes: A dictionary containing process information.
        """
        super(ProductionModel, self).__init__()

        # save data in the model instance
        self.products = products
        self.resources = resources
        self.processes = processes

        # flag to monitor solution status
        self.solved = False

    def load_data(self):
        """
        Prepare the data and pass the information to AMPL.
        """
        # convert the data dictionaries into pandas data frames
        products = pd.DataFrame(self.products).T
        products.rename(columns={'price':'cp'}, inplace=True)
        products.fillna(INF, inplace=True)
        products.index.rename("PRODUCTS", inplace=True)

        resources = pd.DataFrame(self.resources).T
        resources.rename(columns={'price':'cr'}, inplace=True)
        resources.fillna(INF, inplace=True)
        resources.index.rename("RESOURCES", inplace=True)

        processes = pd.DataFrame(self.processes).T

        # display the generated data frames
        display(products)
        display(resources)
        display(processes)

        # pass data to AMPL
        self.set_data(products, "PRODUCTS")
        self.set_data(resources, "RESOURCES")
        self.get_parameter("a").set_values(processes.unstack())

    def solve(self, solver=SOLVER):
        """
        Read the model, load the data, set the solver and solve the optimization problem.
        """
        self.read("production_planning.mod")
        self.load_data()
        self.option["solver"] = solver
        super(ProductionModel, self).solve()
        self.solved = True

    def report(self):
        """
        Solve, if necessary, then report the model solution.
        """
        if not self.solved:
            self.solve()

        print(f"Profit = {self.obj['profit'].value()}")

        print("\nProduction Report")
        y = self.var["y"].get_values().to_pandas()
        y.rename(columns={y.columns[0]: "produced"}, inplace=True)
        y.index.rename("PRODUCTS", inplace=True)
        display(y)

        print("\nResource Report")
        x = self.var["x"].get_values().to_pandas()
        x.rename(columns={x.columns[0]: "consumed"}, inplace=True)
        x.index.rename("RESOURCES", inplace=True)
        display(x)


m = ProductionModel(products, resources, processes)
m.report()


,cp,demand
PRODUCTS,,
U,270.0,40.0
V,210.0,inf


,cr,available
RESOURCES,,
M,10.0,inf
labor A,50.0,100.0
labor B,40.0,80.0


,M,labor A,labor B
U,10,2,1
V,9,1,1


HiGHS 1.5.1: HiGHS 1.5.1: optimal solution; objective 2400
3 simplex iterations
0 barrier iterations
Profit = 2400.0

Production Report


,produced
PRODUCTS,
U,20.0
V,60.0



Resource Report


,consumed
RESOURCES,
M,740.0
labor A,100.0
labor B,80.0
